### processing inkml as images:

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import inkml2img_pictures as ink
import os
import time

#### parsing:
I put the files into a folder called math and the file unzipped is the ICFHR_package. (I manually unzipped that one). Note, there are zipped folders inside the zipped folder lol

In [10]:
#made a directory named math first and unzipped ICFHR to that
path = "./math/ICFHR_package/"
os.listdir(path)

['crohme.css',
 'CROHME2011_data',
 'CROHME2012_data',
 'CROHME_papers',
 'evaluationTools',
 'index.html',
 'InkmlViewer',
 'ParticipantsResults2012']

In [94]:
hehe = np.arange(2 * 10).reshape(2,-1)
hehe

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]])

In [95]:
bias = np.ones((2,1))
bias

array([[1.],
       [1.]])

In [96]:
np.add(hehe, bias)

array([[ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.],
       [11., 12., 13., 14., 15., 16., 17., 18., 19., 20.]])

In [11]:
os.listdir(path + 'CROHME2011_data/CROHME_test')

['CROHME_test.zip']

In [22]:
#all locations of inkml files:

train_2011 = path + "CROHME2011_data/CROHME_training/CROHME_training/"
train_2012 = path + "CROHME2012_data/trainData/trainData/"

test_2011 = path + "CROHME2011_data/CROHME_test/CROHME_test/"
test_2012 = path + "CROHME2012_data/testData/testData/"

testGT_2011 = path + "CROHME2011_data/CROHME_test/CROHME_test/"
testGT_2012 = path + "CROHME2012_data/testDataGT/" #notice, slighlty different path...

paths = [train_2011, train_2012, test_2011, test_2012, testGT_2011, testGT_2012]
paths

['./math/ICFHR_package/CROHME2011_data/CROHME_training/CROHME_training/',
 './math/ICFHR_package/CROHME2012_data/trainData/trainData/',
 './math/ICFHR_package/CROHME2011_data/CROHME_test/CROHME_test/',
 './math/ICFHR_package/CROHME2012_data/testData/testData/',
 './math/ICFHR_package/CROHME2011_data/CROHME_test/CROHME_test/',
 './math/ICFHR_package/CROHME2012_data/testDataGT/']

### unzipping
lots of files are zipped (train files) so this is to unzip them:

In [19]:
import zipfile

for file in paths[:-1]: #for some reason testGT_2012 is not zipped??? Idk why but just exclude that one in unzipping
    with zipfile.ZipFile(file[:-1] + ".zip", 'r') as zip_ref:
        zip_ref.extractall(file)

##### getting image paths:

In [25]:
#dictionary with all file paths to inkml files (some other files present, so be warry, not all inkmls, just what's in file)
ink_names = {}
for ink_file in paths:
    ind_paths = [ink_file + (ink).strip(".inkml") for ink in os.listdir(ink_file)]
    ink_names[ink_file] = ind_paths


## converting images:

In [31]:
#making new folders for converted images:
folders =  ["./train_data_2011", "./train_data_2012", "./test_data_2011", "./test_data_2012", "./testGT_data_2011", "./testGT_data_2012"]

for direct in folders:
    try:
         os.mkdir(direct)
    except:
        print("this directory already exists, '{}', come up with a new one or perish".format(direct))


this directory already exists, './train_data_2011', come up with a new one or perish
this directory already exists, './train_data_2012', come up with a new one or perish
this directory already exists, './test_data_2011', come up with a new one or perish
this directory already exists, './test_data_2012', come up with a new one or perish
this directory already exists, './testGT_data_2011', come up with a new one or perish
this directory already exists, './testGT_data_2012', come up with a new one or perish


In [63]:
#example:
for places in zip(folders, paths):
    print(places[0], places[1])
    
#example getting 1 image, generalized below:
# [ink.inkml2img('./{}.inkml'.format(file_name), train_2011_folder + '/{}.png'.format(ink_name.strip(".inkml"))) for file_name, ink_name in zip(paths, train_2011_ink)]


./train_data_2011 ./math/ICFHR_package/CROHME2011_data/CROHME_training/CROHME_training/
./train_data_2012 ./math/ICFHR_package/CROHME2012_data/trainData/trainData/
./test_data_2011 ./math/ICFHR_package/CROHME2011_data/CROHME_test/CROHME_test/
./test_data_2012 ./math/ICFHR_package/CROHME2012_data/testData/testData/
./testGT_data_2011 ./math/ICFHR_package/CROHME2011_data/CROHME_test/CROHME_test/
./testGT_data_2012 ./math/ICFHR_package/CROHME2012_data/testDataGT/


just as a heads up, lots of warnings so I suppressed, will take quite a bit of time though. Also didn't run fully on my own computer, but there's a lot of files hehe so should be fun. Maybe zip after???

In [85]:
import warnings
#there's a lot of warnings based on below... just as a warning haha
warnings.filterwarnings('ignore')

In [99]:
%%time
bad_links = []

#converting images:
for places in zip(folders, paths):
    #hold onto labels
    traces_dict = {}
    
    print(places[0], places[1])
    new_directory = places[0]
    inkml_directory = places[1]
    
    #I don't know how to parallelize this or if it's even worth it tbh since we're only doing it once:
    for ink_name in ink_names[inkml_directory]:#ink_names[inkml_directory]:
        try:
            file_name = inkml_directory
            #print(file_name, ink_name)
            #print(new_directory)
            #process image:
            ink.inkml2img('{}.inkml'.format(ink_name), output_path = new_directory  + '/{}.png'.format(ink_name.strip(".inkml").split("/")[-1]))

            #get traces:
            trace_data = ink.get_traces_data(ink_name + ".inkml")
            traces_dict[ink_name] = trace_data
        except:
            bad_links.append(ink_name)
#             print(exception_output)
            print(ink_name)
    traces_labels = pd.Series(traces_dict)
    traces_labels.to_csv(path_or_buf = new_directory + "/" + str(new_directory) + "_traces.csv", index = True, header = False)
    pd.Series(bad_links).to_csv("bad_links.csv")
        


./train_data_2011 ./math/ICFHR_package/CROHME2011_data/CROHME_training/CROHME_training/
./train_data_2012 ./math/ICFHR_package/CROHME2012_data/trainData/trainData/
./math/ICFHR_package/CROHME2012_data/trainData/trainData/tegral05
./test_data_2011 ./math/ICFHR_package/CROHME2011_data/CROHME_test/CROHME_test/
./math/ICFHR_package/CROHME2011_data/CROHME_test/CROHME_test/tegral01
./test_data_2012 ./math/ICFHR_package/CROHME2012_data/testData/testData/
./testGT_data_2011 ./math/ICFHR_package/CROHME2011_data/CROHME_test/CROHME_test/
./math/ICFHR_package/CROHME2011_data/CROHME_test/CROHME_test/tegral01
./testGT_data_2012 ./math/ICFHR_package/CROHME2012_data/testDataGT/
./math/ICFHR_package/CROHME2012_data/testDataGT/statSR.txt
Wall time: 20min 25s


<Figure size 432x288 with 0 Axes>

#### getting traces:

just sample code to see how to get traces and output:

In [42]:
sample1 = ink_names[paths[0]][0]
sample1
sample2 = ink_names[paths[0]][1]
sample2

'./math/ICFHR_package/CROHME2011_data/CROHME_training/CROHME_training/formulaire001-equation003'

In [54]:
hi = pd.Series({0: ink.get_traces_data(sample1 + ".inkml"), 1: ink.get_traces_data(sample2 + ".inkml")})
hi.index = ink_names[paths[0]][:2]
hi

./math/ICFHR_package/CROHME2011_data/CROHME_training/CROHME_training/formulaire001-equation001    [{'label': '\phi', 'trace_group': [[[117004, 1...
./math/ICFHR_package/CROHME2011_data/CROHME_training/CROHME_training/formulaire001-equation003    [{'label': 'c', 'trace_group': [[[76157, 25865...
dtype: object